# CMFGEN

Database from John Hillier’s CMFGEN, a radiative transfer code designed to solve the radiative transfer and statistical equilibrium equations in spherical geometry.

<div class="alert alert-info">

**Note:**
    
In this example, the data was downloaded from the [CMFGEN website](http://kookaburra.phyast.pitt.edu/hillier/web/CMFGEN.htm) and extracted to the `/tmp/atomic` folder.
    
</div>



## Parsers

The CMFGEN parsers read data from text files and retrieves the information as DataFrames. Currently `osc`, `col` and `pho` files are supported.


### Levels, Lines and Collisions

Energy levels and spectral lines are stored in the `osc` files, while collisions strengths are kept in the `col` files.

In [ ]:
from carsus.io.cmfgen import CMFGENEnergyLevelsParser, CMFGENOscillatorStrengthsParser, CMFGENCollisionalStrengthsParser

In [ ]:
si2_lvl = CMFGENEnergyLevelsParser('/tmp/atomic/SIL/II/16sep15/si2_osc_kurucz')

In [ ]:
si2_osc = CMFGENOscillatorStrengthsParser('/tmp/atomic/SIL/II/16sep15/si2_osc_kurucz')

In [ ]:
si2_col = CMFGENCollisionalStrengthsParser('/tmp/atomic/SIL/II/16sep15/si2_col')

The header information is stored in the `meta` attribute, for example:

In [ ]:
si2_col.meta

And the table is stored in the `base` attribute:

In [ ]:
si2_lvl.base

Finally, dump the data with the `to_hdf` method.

### Photoionization Cross-sections

Photoionization cross-sections are stored in the `pho` files.

In [ ]:
from carsus.io.cmfgen import CMFGENPhotoionizationCrossSectionParser

In [ ]:
si2_cross_sections = CMFGENPhotoionizationCrossSectionParser('/tmp/atomic/SIL/II/16sep15/phot_nahar_A')

In [ ]:
si2_cross_sections.meta

In this case, the `base` attribute is a list containing many DataFrames.

In [ ]:
type(si2_cross_sections.base)

In [ ]:
len(si2_cross_sections.base)

There are three different types of photoionization cross-sections tables: points, fit coefficients and Verner analytic fits for the ground state shells (not shown in this example).

In [ ]:
si2_cross_sections.base[0]

In [ ]:
si2_cross_sections.base[92]

### Batch Convert Files to HDF5

To convert multiple CMFGEN files to the HDF5 format import the `hdf_dump` function.

In [ ]:
from carsus.io.cmfgen import hdf_dump

In [ ]:
hdf_dump('/tmp/atomic/', ['osc', 'OSC', 'Osc'], CMFGENEnergyLevelsParser(), chunk_size=10, ignore_patterns=['ERROR_CHK'])

Required parameters are `cmfgen_dir`, `patterns` and `parser`, while `chunk_size` and `ignore_patterns` are optional.

## CMFGENReader

The `CMFGENReader` mimics the structure of `GFALLReader` and provides `levels` and `lines` tables for the selected ions.

In [ ]:
from carsus.io.cmfgen import CMFGENReader

<div class="alert alert-info">

**Note:**

Remember in Carsus `Si 0` is Si I, `Si 1` is Si II, etc.
    
See [Notation in Carsus](../development/notation.rst).
    
</div>

In [ ]:
cmfgen_data = {'Si 1': {'levels': si2_lvl, 'lines': si2_osc}, }
cmfgen_reader = CMFGENReader(cmfgen_data, priority=20)

In [ ]:
cmfgen_reader.levels

In [ ]:
cmfgen_reader.lines